In [34]:
import seaborn as sns
import pandas as pd
import numpy as np

# Pandas Combining DataFrames

In pandas there are 4 (plus a few special case) ways to combine data from different frames:

* Merging
* Joining
* Concatenating 
* Appending

Where merging and joining are basically redundant and concatenating and appending are basically redundant. 

So today we will be going over Merging and Concatenating in pandas. 

Check out the full documentation [here](http://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html), but be warned it is a bit long :)


Okay let's get started.

In [35]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


## Merge

Merging is for doing complex column-wise combinations of dataframes in a SQL-like way. If you don't know SQL joins then check out this resource [sql joins](https://www.w3schools.com/sql/sql_join.asp) and comment below (and maybe I'll make a video). 

Two merge we need two dataframes, let's make them below:

In [36]:
tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

tips_bill

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [37]:
del tips_bill['tip']
del tips_tip['total_bill']

In [38]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [39]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

Now that we have two datasets that we want to combine (aka take the tips and combine with the total bill), how do we do it? We merge!

In [8]:
# pd.merge()

Notice that there are a ton of options:

In [40]:
pd.merge(
tips_bill, tips_tip)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [41]:
# we can merge on the indexes
pd.merge(tips_bill, tips_tip, 
         right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [42]:
tips_bill.reset_index()

,sex,smoker,total_bill
0,Male,Yes,1337.07
1,Male,No,1919.75
2,Female,Yes,593.27
3,Female,No,977.68


In [43]:
#we can reset indexes and then merge on the columns - perhaps the easiest way
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(),
    on=['sex', 'smoker']
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [44]:
# it can actually infer the above - but be very careful with this
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index()
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [28]:
# it can merge on partial column and index
pd.merge(
    tips_bill.reset_index(), 
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [18]:
tips_bill.reset_index()

,sex,smoker,total_bill
0,Male,Yes,1337.07
1,Male,No,1919.75
2,Female,Yes,593.27
3,Female,No,977.68


In [17]:
tips_tip.reset_index()

,sex,smoker,tip
0,Male,Yes,183.07
1,Male,No,302.00
2,Female,Yes,96.74
3,Female,No,149.77


In [45]:
# we can do any SQL-like functionality
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='left'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [19]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='inner'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00


In [21]:
# and if you add an indicator...
pd.merge(
    tips_bill.reset_index().tail(3), 
    tips_tip.reset_index().head(3),
    how='outer',
    indicator=True
)

,sex,smoker,total_bill,tip,_merge
0,Male,No,1919.75,302.00,both
1,Female,Yes,593.27,96.74,both
2,Female,No,977.68,NaN,left_only
3,Male,Yes,NaN,183.07,right_only


In [22]:
# it can handle columns with the same name
pd.merge(tips_bill, 
         tips_bill, 
         right_index=True, 
         left_index=True,
         suffixes=('_left', '_right')
)

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

This is one of the most complex parts of pandas - but it is very important to master. So please do check out the excerises below!

One thing to be careful with here is merging two data types. Strings are not equal to ints!

# Contatenation

Concatenating is for combining more than two dataframes in either column-wise or row-wise. The problem with concatenate is that the combinations it allows you to do are rather simplistic. That's why we need merge. 

Concatenate can take as many data frames as you want, but it requires that they are specifically constructed. All of the dataframes you pass in will need to have the same index. So no more using columns as an index. 

Let's check out basic use below:

In [48]:
# this adds the dataframes together row wise

pd.concat([tips_bill,  tips_tip, tips_tip])

# can we add one more dataframe?

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [47]:
# this does it column wise
pd.concat([tips_bill, tips_tip], axis=1)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [29]:
# and finally this will add on the dataset where it's from
pd.concat([tips_bill, tips_tip],keys=['num0', 'num1'])

total_bill     tip
     sex    smoker                    
num0 Male   Yes        1337.07     NaN
            No         1919.75     NaN
     Female Yes         593.27     NaN
            No          977.68     NaN
num1 Male   Yes            NaN  183.07
            No             NaN  302.00
     Female Yes            NaN   96.74
            No             NaN  149.77

As you can see there is not a ton of functionality to concat, but it is invaluable if you have more than one dataframe or you are looking to append the rows of one dataframe onto another.

## Conclusion

There are a couple of other ways to merge data, but they are pretty niche (and mainly for time series data). If y'all have a desire for me to go over them then comment below!

They are:

* combine_first
* merge_ordered
* merge_asof

Otherwise you should be fully equipped to do the [exercises](https://github.com/guipsamora/pandas_exercises#merge). These functions require a bit of practice to get used to, so don't be discouraged if it takes some time.